In [1]:
import altair as alt
import pandas as pd
# saving data into a file rather than embedding into the chart
alt.data_transformers.enable('json') 
# alt.renderers.enable('notebook')
# !pip install geopandas
import geopandas as gpd
import json
# !pip install bokeh==1.1
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.models import Slider, HoverTool
from bokeh.models import ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import RdYlBu5,RdGy5
import random
from bokeh.models.widgets import CheckboxGroup
from bokeh.models.widgets import Select
from bokeh.plotting import figure, curdoc,show
from bokeh.layouts import column, row, WidgetBox
from bokeh.models import Panel
from bokeh.models.widgets import Tabs
from bokeh.models.widgets import CheckboxGroup

In [2]:
data_visatype = pd.read_csv('country_preprocessed.csv')
data_years_data=pd.read_csv('country_rank_score.csv')
centroid_data=pd.read_csv('country_centroids_az8.csv')
income_gdp=pd.read_csv('IncomingGDPData.csv')
hd=pd.read_csv('highlight differences.csv')

In [3]:
data_years_data.head()
data_years_data['year'].unique()

array([2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019], dtype=int64)

In [4]:
sub_rank_2018=data_years_data[data_years_data['year']==2018]

sub_rank_2018.shape
sub_rank_2018.head()

sub_rank_2018['country_name'].unique()
sub_rank_2018.head()

,id,passport_id,rank,score,year,country_name,code
2354,216,6,1,190.0,2018,Japan,JPN
2355,218,8,2,189.0,2018,Singapore,NaN
2356,211,1,3,188.0,2018,Germany,DEU
2357,214,4,3,188.0,2018,France,FRA
2358,2118,18,3,188.0,2018,South Korea,KOR


In [5]:
data_visatype.head()

,country1,country2,code1,code2,visa_type,lat2,lng2,lat1,lng1
0,Afghanistan,Afghanistan,AFG,AFG,5,34.5167,69.1833,34.5167,69.1833
1,Afghanistan,Albania,AFG,ALB,4,41.3275,19.8189,34.5167,69.1833
2,Afghanistan,Algeria,AFG,DZA,4,36.7631,3.0506,34.5167,69.1833
3,Afghanistan,Andorra,AFG,NaN,4,42.5000,1.5165,34.5167,69.1833
4,Afghanistan,Angola,AFG,AGO,4,-8.8383,13.2344,34.5167,69.1833


In [6]:
# result_data = pd.merge(sub_rank_2018,data_visatype[['country1','lat1', 'lng1', 'lat2','lng2']],
#                   left_on='country_name', right_on='country1',
#                  how='left')




In [7]:
shapefile = 'ne_110m_admin_0_countries.shp'
#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
#Rename columns.
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head()

,country,country_code,geometry
0,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555..."
1,United Republic of Tanzania,TZA,POLYGON ((33.90371119710453 -0.950000000000000...
2,Western Sahara,SAH,POLYGON ((-8.665589565454809 27.65642588959236...
3,Canada,CAN,"(POLYGON ((-122.84 49.00000000000011, -122.974..."
4,United States of America,USA,"(POLYGON ((-122.84 49.00000000000011, -120 49...."


In [8]:
# df4 = sub_rank_2018.copy()

# def fun(col):
#     for i in list(sub_rank_2018['country_name'].unique()):
#       if str(col) in str(i) or str(i) in str(col) :
#         return i


# gdf['country_name'] =  gdf['country'].apply(fun) 
merged = gdf.merge(sub_rank_2018, left_on='country_code',right_on='code',how='inner')



In [9]:
# merged = gdf.merge(sub_rank_2018, left_on = 'country', right_on = 'country_name',how='inner')
merged.shape
merged.head()

,country,country_code,geometry,id,passport_id,rank,score,year,country_name,code
0,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",21100,100,56,90.0,2018,Fiji,FJI
1,United Republic of Tanzania,TZA,POLYGON ((33.90371119710453 -0.950000000000000...,21123,123,74,68.0,2018,Tanzania,TZA
2,Canada,CAN,"(POLYGON ((-122.84 49.00000000000011, -122.974...",2121,21,6,185.0,2018,Canada,CAN
3,United States of America,USA,"(POLYGON ((-122.84 49.00000000000011, -120 49....",2120,20,5,186.0,2018,United States,USA
4,Kazakhstan,KAZ,"POLYGON ((87.35997033076265 49.21498078062912,...",21114,114,69,76.0,2018,Kazakhstan,KAZ


In [10]:
merged.head()
merged['country_name'].isnull().sum()
# merged['score'].describe()

merged[merged.country_name.isnull()]['country']

Series([], Name: country, dtype: object)

In [11]:
#Read data to json.
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)
output_notebook()

Loading BokehJS ...

In [13]:
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest score.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 190)
#Define custom tick labels for color bar.
tick_labels = {'1': '30', '2': '50', '3':'80', '4':'100', '5':'120', '6':'150', '7':'>190+'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

#Add hover tool
hover = HoverTool(tooltips = [ ('Country/region','@country'),('% score', '@score')])

#Create figure object.
p = figure(title = 'Travel visa Inequality Score', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None



#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'score', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
show(p)
# curdoc().add_root(p)


In [14]:
income_gdp.head()


,connections,country_name,code,gdp_percapita,incoming,iso3_digit_code,lat,long,outgoing
0,"['Sri Lanka', 'Malaysia', 'Algeria', 'Dominica...",Yemen,YEM,1473.10,990000,887,15.552727,48.516388,0
1,"['Barbados', 'Peru', 'Bahamas', 'Hong Kong', '...",Fiji,FJI,7750.43,658000,242,-16.578193,179.414413,0
2,"['Barbados', 'Canada', 'Dominica', 'United Kin...",Brunei,BRN,71776.65,225000,96,4.535277,114.727669,0
3,"['Peru', 'Panama', 'Argentina', 'Bahamas', 'Ir...",Bolivia,BOL,6131.06,798000,68,-16.290154,-63.588653,837000
4,"['Kenya', 'Kuwait', 'Venezuela', 'Malaysia', '...",Cambodia,KHM,1006.84,2882000,116,12.565679,104.990963,710000


In [15]:
# merged2.head()

In [16]:
# merged[merged['country']=='Papua New Guinea']
merged2=merged.merge(centroid_data[['gu_a3','Longitude','Latitude']],how='inner',left_on='country_code',right_on='gu_a3')
merged3=merged2.merge(income_gdp,how='left',left_on='country_code',right_on='code')

In [17]:
merged2.head()

,country,country_code,geometry,id,passport_id,rank,score,year,country_name,code,gu_a3,Longitude,Latitude
0,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",21100,100,56,90.0,2018,Fiji,FJI,FJI,165.451954,-17.428580
1,United Republic of Tanzania,TZA,POLYGON ((33.90371119710453 -0.950000000000000...,21123,123,74,68.0,2018,Tanzania,TZA,TZA,34.813100,-6.275654
2,Canada,CAN,"(POLYGON ((-122.84 49.00000000000011, -122.974...",2121,21,6,185.0,2018,Canada,CAN,CAN,-98.307770,61.362063
3,United States of America,USA,"(POLYGON ((-122.84 49.00000000000011, -120 49....",2120,20,5,186.0,2018,United States,USA,USA,-112.461674,45.679547
4,Kazakhstan,KAZ,"POLYGON ((87.35997033076265 49.21498078062912,...",21114,114,69,76.0,2018,Kazakhstan,KAZ,KAZ,67.291494,48.156881


In [18]:
merged3['gdp_millions']=merged3['gdp_percapita']/1000000
merged3['norm_percapita']=(merged3['gdp_percapita']-merged3['gdp_percapita'].min())/(merged3['gdp_percapita'].max()-merged3['gdp_percapita'].min())

merged3['norm_percapita']=merged3['norm_percapita']*40

merged3['norm_outgoing']=(merged3['outgoing']-merged3['outgoing'].min())/(merged3['outgoing'].max()-merged3['outgoing'].min())

merged3['norm_outgoing']=merged3['norm_outgoing']*40

merged3['norm_incoming']=(merged3['incoming']-merged3['incoming'].min())/(merged3['incoming'].max()-merged3['incoming'].min())

merged3['norm_incoming']=merged3['norm_incoming']*40

In [19]:
#Read data to json.
merged_json = json.loads(merged3.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

In [20]:
output_notebook()

Loading BokehJS ...

In [21]:
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest score.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 190, nan_color = '#d9d9d9')
#Define custom tick labels for color bar.
tick_labels = {'1': '30', '2': '50', '3':'80', '4':'100', '5':'120', '6':'150', '7':'>190+'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

#Add hover tool
hover = HoverTool(tooltips = [ ('Country/region','@country'),('% score', '@score'),('GDP','@gdp_millions')])

#Create figure object.
p1 = figure(title = 'Travel visa Inequality Score and GDP', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p1.patches('xs','ys', source = geosource,fill_color = {'field' :'score', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p1.add_layout(color_bar, 'below')
p1.circle('Longitude','Latitude',size='norm_percapita', color="orange", alpha=0.7,source=geosource)
show(p1)
# curdoc().add_root(p1)



In [22]:
output_notebook()
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest score.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 190, nan_color = '#d9d9d9')
#Define custom tick labels for color bar.
tick_labels = {'1': '30', '2': '50', '3':'80', '4':'100', '5':'120', '6':'150', '7':'>190+'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

#Add hover tool
hover = HoverTool(tooltips = [ ('Country/region','@country'),('% score', '@score'),('outgoing','@outgoing')])

#Create figure object.
p2 = figure(title = 'Travel visa inequality score and Incoming/Outgoing People', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
p2.xgrid.grid_line_color = None
p2.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p2.patches('xs','ys', source = geosource,fill_color = {'field' :'score', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p2.add_layout(color_bar, 'below')
p2.circle('Longitude','Latitude',size='norm_incoming', color="orange", alpha=0.7,source=geosource)
p2.circle('Longitude','Latitude',size='norm_outgoing', color="red", alpha=0.7,source=geosource)
show(p2)
# curdoc().add_root(p2)


Loading BokehJS ...

In [23]:
#data_visatype.head()
#data_visatype['visa_type'].unique()

def visa_cat(col):
  if col==0:
    return 'Visa Not Required'
  elif col==1:
    return 'Visa On Arrival'
  elif col==2:
    return 'ETA Required'
  elif col==3:
    return 'eVisa Required'
  elif col==4:
    return 'Visa Required'
  elif col==5:
    return 'Admission Refused'

data_visatype['cat']=data_visatype['visa_type'].apply(visa_cat)



In [24]:
# print(color_mapper)
data_visatype[data_visatype['country2']=='China'].head()

,country1,country2,code1,code2,visa_type,lat2,lng2,lat1,lng1,cat
33,Afghanistan,China,AFG,CHN,4,39.9289,116.3883,34.5167,69.1833,Visa Required
216,Albania,China,ALB,CHN,4,39.9289,116.3883,41.3275,19.8189,Visa Required
399,Algeria,China,DZA,CHN,4,39.9289,116.3883,36.7631,3.0506,Visa Required
582,Andorra,China,NaN,CHN,4,39.9289,116.3883,42.5000,1.5165,Visa Required
765,Angola,China,AGO,CHN,4,39.9289,116.3883,-8.8383,13.2344,Visa Required


In [24]:
#gdf[gdf['country']=='China'].head()
# gdf.head()

In [25]:
#data_visatype.head()
vis2= gdf.merge(data_visatype, left_on='country_code',right_on='code2',how='inner')

In [27]:
vis2.head()
import pickle
file=open('df.pickle','wb')
pickle.dump(vis2[['code1','code2','visa_type']],file)
# vis3=vis2[vis2['country1'] == 'Canada']
# vis3.head()

In [27]:
vis2.shape

(29463, 13)

In [29]:
# #Read data to json.
# merged_json = json.loads(vis3.to_json())
# #Convert to String like object.
# json_data2 = json.dumps(merged_json)

In [30]:

# vis3['geometry'].head(1)
# output_notebook()

In [31]:
# #Input GeoJSON source that contains features for plotting.
# geosource = GeoJSONDataSource(geojson = json_data2)


# categ=['Visa Not Required','Visa On Arrival','ETA Required','eVisa Required','Visa Required','Admission Refused']
# color_mapper2 = CategoricalColorMapper(factors=categ, palette=[RdYlBu5[0], RdYlBu5[1],RdYlBu5[2],RdYlBu5[3],RdYlBu5[4],RdGy5[0]])



# # #Create color bar. 
# # color_bar = ColorBar(color_mapper=color_mapper2)
# # ColorBar(color_mapper=color_mapper2 , label_standoff=12, location=(0,0))

# #Add hover tool
# hover = HoverTool(tooltips = [ ('Country/region','@country'),('travel category', '@cat')])

# #Create figure object.
# p2 = figure(title = ('Where can a selected Country Travel to?'), plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
# p2.xgrid.grid_line_color = None
# p2.ygrid.grid_line_color = None

# #Add patch renderer to figure. 
# p2.patches('xs','ys', source = geosource,fill_color = {'field' :'cat', 'transform' : color_mapper2}, line_color = 'black', line_width = 0.25, fill_alpha = 1)

# for factor, color in zip(color_mapper2.factors, color_mapper2.palette):
#     p2.circle(x=[], y=[], fill_color=color, legend=factor)
# # show(p2)
# #Specify figure layout.
# # p.add_layout(color_bar, 'below')




In [32]:
# data1=GeoJSONDataSource(geojson = json_data2)

In [28]:
# vis2.head()
# p_df.head()

# # !pip install  geopandas
# import geopandas
from shapely.geometry import Point, LineString, Polygon
from bokeh.models.widgets import Select
from bokeh.layouts import widgetbox, row, column
output_notebook()

Loading BokehJS ...

In [29]:
#method 3:


def make_dataset(cat_list):
  subset = vis2[vis2['country1'] == cat_list]
  merged_json1 = json.loads(subset.to_json())
  json_data2 = json.dumps(merged_json1)
  return json_data2

def update_plot(attr, old, new):
    # The input yr is the year selected from the slider
    country = select.value
    new_data = make_dataset(country)
    
    # Update the plot based on the changed inputs
    p3 = make_plot()
    
    # Update the layout, clear the old document and display the new document
    layout = column(widgetbox(select),p3)
#     curdoc().clear()
#     curdoc().add_root(layout)
    
    # Update the data
    geosource.geojson = new_data 
def make_plot():
  categ=['Visa Not Required','Visa On Arrival','ETA Required','eVisa Required','Visa Required','Admission Refused']
  color_mapper2 = CategoricalColorMapper(factors=categ, palette=[RdYlBu5[0], RdYlBu5[1],RdYlBu5[2],RdYlBu5[3],RdYlBu5[4],RdGy5[0]])

  #Define custom tick labels for color bar.
  tick_labels1 = {'0': 'Visa Not Required', '1': 'Visa On arrival', '3':'ETA Required', '4':'Visa Required', '5':'Admission Refused'}

  # #Create color bar. 
  # color_bar = ColorBar(color_mapper=color_mapper2)
  # ColorBar(color_mapper=color_mapper2 , label_standoff=12, location=(0,0))

  #Add hover tool
  hover = HoverTool(tooltips = [ ('Country/region','@country'),('travel category', '@cat')])

  #Create figure object.
  p3 = figure(title = ('Where can a selected Country Travel to?'), plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
  p3.xgrid.grid_line_color = None
  p3.ygrid.grid_line_color = None

  #Add patch renderer to figure. 
  p3.patches('xs','ys', source = geosource,fill_color = {'field' :'cat', 'transform' : color_mapper2}, line_color = 'black', line_width = 0.25, fill_alpha = 1)

  for factor, color in zip(color_mapper2.factors, color_mapper2.palette):
      p3.circle(x=[], y=[], fill_color=color, legend=factor)
  #Specify figure layout.
  return p3

geosource = GeoJSONDataSource(geojson =make_dataset('Canada'))
p3 = make_plot()
countries=list(vis2['country1'].unique())


select = Select(title='Selected Country:', value='Canada', options=countries)
select.on_change('value', update_plot)

layout = column(widgetbox(select),p3)
show(layout)
# curdoc().add_root(layout)

# Use the following code to test in a notebook, comment out for transfer to live site
# Interactive features will not show in notebook
#output_notebook()
# show(layout)
# final_layout = column(p,p1,p2,layout)
# curdoc().add_root(final_layout)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [30]:
hd.head()

,code1,code2,visa_type,type2,diff
0,AFG,AFG,5,5,2
1,AFG,ALB,4,4,1
2,AFG,DZA,4,4,1
3,AFG,NaN,4,0,-1
4,AFG,AGO,4,4,1


In [31]:
country_poly = vis2.loc[vis2['country'].drop_duplicates(keep='last').index]

In [32]:
country_poly.head()

,country,country_code,geometry,country1,country2,code1,code2,visa_type,lat2,lng2,lat1,lng1,cat
182,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Zimbabwe,Fiji,ZWE,FJI,0,-18.1330,178.4417,-17.8178,31.0447,Visa Not Required
548,United Republic of Tanzania,TZA,POLYGON ((33.90371119710453 -0.950000000000000...,Zimbabwe,Tanzania,ZWE,TZA,0,-6.8000,39.2683,-17.8178,31.0447,Visa Not Required
731,Canada,CAN,"(POLYGON ((-122.84 49.00000000000011, -122.974...",Zimbabwe,Canada,ZWE,CAN,4,45.4167,-75.7000,-17.8178,31.0447,Visa Required
914,United States of America,USA,"(POLYGON ((-122.84 49.00000000000011, -120 49....",Zimbabwe,United States,ZWE,USA,4,38.9047,-77.0163,-17.8178,31.0447,Visa Required
1097,Kazakhstan,KAZ,"POLYGON ((87.35997033076265 49.21498078062912,...",Zimbabwe,Kazakhstan,ZWE,KAZ,4,51.1811,71.4278,-17.8178,31.0447,Visa Required


In [33]:
hd.head()

,code1,code2,visa_type,type2,diff
0,AFG,AFG,5,5,2
1,AFG,ALB,4,4,1
2,AFG,DZA,4,4,1
3,AFG,NaN,4,0,-1
4,AFG,AGO,4,4,1


In [68]:
vis2

,country,country_code,geometry,country1,country2,code1,code2,visa_type,lat2,lng2,lat1,lng1,cat
0,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Afghanistan,Fiji,AFG,FJI,4,-18.133,178.4417,34.5167,69.1833,Visa Required
1,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Albania,Fiji,ALB,FJI,4,-18.133,178.4417,41.3275,19.8189,Visa Required
2,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Algeria,Fiji,DZA,FJI,4,-18.133,178.4417,36.7631,3.0506,Visa Required
3,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Andorra,Fiji,NaN,FJI,4,-18.133,178.4417,42.5000,1.5165,Visa Required
4,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Angola,Fiji,AGO,FJI,4,-18.133,178.4417,-8.8383,13.2344,Visa Required
5,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Argentina,Fiji,ARG,FJI,0,-18.133,178.4417,-34.6025,-58.3975,Visa Not Required
6,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Armenia,Fiji,ARM,FJI,4,-18.133,178.4417,40.1812,44.5136,Visa Required
7,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Australia,Fiji,AUS,FJI,0,-18.133,178.4417,-35.2830,149.1290,Visa Not Required
8,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Austria,Fiji,AUT,FJI,0,-18.133,178.4417,48.2000,16.3666,Visa Not Required
9,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Azerbaijan,Fiji,AZE,FJI,4,-18.133,178.4417,40.3953,49.8622,Visa Required


In [65]:
visa3=vis2[['country_code','geometry']]

visa4=visa3.drop_duplicates(keep='first', inplace=False)

vis5= visa4.merge(hd, left_on='country_code',right_on='code2',how='inner')

In [73]:
temp_data=vis2[['country1','code1']].drop_duplicates(keep='first',inplace=False)

In [74]:
vis6 = vis5.merge(temp_data[['country1','code1']], on='code1',how='inner')

In [76]:
vis6.head()

,country_code,geometry,code1,code2,visa_type,type2,diff,country1
0,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",AFG,FJI,4,4,1,Afghanistan
1,TZA,POLYGON ((33.90371119710453 -0.950000000000000...,AFG,TZA,4,4,1,Afghanistan
2,CAN,"(POLYGON ((-122.84 49.00000000000011, -122.974...",AFG,CAN,4,4,1,Afghanistan
3,USA,"(POLYGON ((-122.84 49.00000000000011, -120 49....",AFG,USA,4,4,1,Afghanistan
4,KAZ,"POLYGON ((87.35997033076265 49.21498078062912,...",AFG,KAZ,4,4,1,Afghanistan


In [26]:
vis2[vis2['country1']=='Germany']

,country,country_code,geometry,country1,country2,code1,code2,visa_type,lat2,lng2,lat1,lng1,cat
56,Fiji,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",Germany,Fiji,DEU,FJI,0,-18.1330,178.4417,52.5218,13.4015,Visa Not Required
295,United Republic of Tanzania,TZA,POLYGON ((33.90371119710453 -0.950000000000000...,Germany,Tanzania,DEU,TZA,1,-6.1833,35.7500,52.5218,13.4015,Visa On Arrival
296,United Republic of Tanzania,TZA,POLYGON ((33.90371119710453 -0.950000000000000...,Germany,Tanzania,DEU,TZA,1,-6.8000,39.2683,52.5218,13.4015,Visa On Arrival
605,Canada,CAN,"(POLYGON ((-122.84 49.00000000000011, -122.974...",Germany,Canada,DEU,CAN,0,45.4167,-75.7000,52.5218,13.4015,Visa Not Required
788,United States of America,USA,"(POLYGON ((-122.84 49.00000000000011, -120 49....",Germany,United States,DEU,USA,2,38.9047,-77.0163,52.5218,13.4015,ETA Required
971,Kazakhstan,KAZ,"POLYGON ((87.35997033076265 49.21498078062912,...",Germany,Kazakhstan,DEU,KAZ,0,51.1811,71.4278,52.5218,13.4015,Visa Not Required
1154,Uzbekistan,UZB,"POLYGON ((55.96819135928291 41.30864166926936,...",Germany,Uzbekistan,DEU,UZB,4,41.3117,69.2949,52.5218,13.4015,Visa Required
1337,Papua New Guinea,PNG,(POLYGON ((141.0002104025918 -2.60015105551566...,Germany,Papua New Guinea,DEU,PNG,1,-9.4647,147.1925,52.5218,13.4015,Visa On Arrival
1520,Indonesia,IDN,(POLYGON ((141.0002104025918 -2.60015105551566...,Germany,Indonesia,DEU,IDN,0,-6.1744,106.8294,52.5218,13.4015,Visa Not Required
1703,Argentina,ARG,(POLYGON ((-68.63401022758323 -52.636370458874...,Germany,Argentina,DEU,ARG,0,-34.6025,-58.3975,52.5218,13.4015,Visa Not Required


In [34]:
vis5= pd.merge(hd,country_poly ,left_on='code2',right_on='code2',how='left',indicator=True,left_index=True)

In [36]:
vis5.head()

,code1_x,code2,visa_type_x,type2,diff,country,country_code,geometry,country1,country2,code1_y,visa_type_y,lat2,lng2,lat1,lng1,cat,_merge
17384,AFG,AFG,5,5,2,Afghanistan,AFG,"POLYGON ((66.51860680528867 37.36278432875879,...",Zimbabwe,Afghanistan,ZWE,4.0,34.5167,69.1833,-17.8178,31.0447,Visa Required,both
21410,AFG,ALB,4,4,1,Albania,ALB,"POLYGON ((21.0200403174764 40.84272695572588, ...",Zimbabwe,Albania,ZWE,4.0,41.3275,19.8189,-17.8178,31.0447,Visa Required,both
14090,AFG,DZA,4,4,1,Algeria,DZA,POLYGON ((-8.684399786809053 27.39574412689601...,Zimbabwe,Algeria,ZWE,4.0,36.7631,3.0506,-17.8178,31.0447,Visa Required,both
29462,AFG,NaN,4,0,-1,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
12992,AFG,AGO,4,4,1,Angola,AGO,(POLYGON ((12.99551720546518 -4.78110320396188...,Zimbabwe,Angola,ZWE,4.0,-8.8383,13.2344,-17.8178,31.0447,Visa Required,both


In [89]:
#visa3=vis2[['country','country_code','geometry','country1']]

#visa4=visa3.drop_duplicates(keep='first', inplace=False)

#vis5= visa4.merge(hd, left_on='country_code',right_on='code1',how='inner')

def adv_tag(col):
    if col==1:
        return 'Advantage'
    elif col==-1:
        return 'Disadvantage'
    elif col==0:
        return 'Neutral'
    else:
        return 'Selected Country'
    
vis6['tag']=vis6['diff'].apply(adv_tag)



        
            


In [77]:
vis6.reset_index(drop=True,inplace=True)

In [84]:
vis5.head()


,country_code,geometry,code1,code2,visa_type,type2,diff,country1
0,FJI,"(POLYGON ((180 -16.06713266364245, 180 -16.555...",AFG,FJI,4,4,1,Afghanistan
1,TZA,POLYGON ((33.90371119710453 -0.950000000000000...,AFG,TZA,4,4,1,Afghanistan
2,CAN,"(POLYGON ((-122.84 49.00000000000011, -122.974...",AFG,CAN,4,4,1,Afghanistan
3,USA,"(POLYGON ((-122.84 49.00000000000011, -120 49....",AFG,USA,4,4,1,Afghanistan
4,KAZ,"POLYGON ((87.35997033076265 49.21498078062912,...",AFG,KAZ,4,4,1,Afghanistan


In [79]:
subset = vis6[vis6['country1'] == 'Canada']
merged_json1 = json.loads(subset.to_json())
json_data2 = json.dumps(merged_json1)

In [90]:
output_notebook()
subset = vis6[vis6['country1'] == 'Canada']
merged_json1 = json.loads(subset.to_json())
json_data2 = json.dumps(merged_json1)
geosource = GeoJSONDataSource(geojson = json_data2)
tag=['Advantage','Neutral','Disadvantage','Selected Country']
color_mapper2 = CategoricalColorMapper(factors=tag, palette=[RdYlBu5[0],RdYlBu5[1],RdYlBu5[4],RdGy5[0]])

hover = HoverTool(tooltips = [ ('Country/region','@country'),('Advantage tag', '@tag')])
p3 = figure(title = ('selected Country Travel to?'), plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
p3.xgrid.grid_line_color = None
p3.ygrid.grid_line_color = None

 
p3.patches('xs','ys', source = geosource,fill_color = {'field' :'tag', 'transform' : color_mapper2}, line_color = 'black', line_width = 0.25, fill_alpha = 1)

for factor, color in zip(color_mapper2.factors, color_mapper2.palette):
    p3.circle(x=[], y=[], fill_color=color, legend=factor)
show(p3)

Loading BokehJS ...

In [92]:
# #method 3:

output_notebook()
# #Create figure object.


def make_dataset(cat_list):
    subset = vis6[vis6['country1'] == cat_list]
    merged_json1 = json.loads(subset.to_json())
    json_data2 = json.dumps(merged_json1)
    return json_data2

def update_plot(attr, old, new):
     # The input yr is the year selected from the slider
    country = select.value
    new_data = make_dataset(country)
    
#     # Update the plot based on the changed inputs
    p3 = make_plot()
    
#     # Update the layout, clear the old document and display the new document
    layout = column(widgetbox(select),p3)
    geosource.geojson = new_data 
def make_plot():
    tag=['Advantage','Neutral','Disadvantage','Selected Country']
    color_mapper2 = CategoricalColorMapper(factors=tag, palette=[RdYlBu5[0],RdYlBu5[1],RdYlBu5[4],RdGy5[0]])

    hover = HoverTool(tooltips = [ ('Country/region','@code2'),('Advantage tag', '@tag')])
    p3 = figure(title = ('selected Country Travel to?'), plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
    p3.xgrid.grid_line_color = None
    p3.ygrid.grid_line_color = None
    

#   #Add patch renderer to figure. 
    p3.patches('xs','ys', source = geosource,fill_color = {'field' :'tag', 'transform' : color_mapper2}, line_color = 'black', line_width = 0.25, fill_alpha = 1)

    for factor, color in zip(color_mapper2.factors, color_mapper2.palette):
        p3.circle(x=[], y=[], fill_color=color, legend=factor)
#   #Specify figure layout.
    return p3

geosource = GeoJSONDataSource(geojson =make_dataset('Canada'))
p4 = make_plot()
countries=list(vis2['country1'].unique())


select = Select(title='Select Country:', value='Canada', options=countries)
#select.on_change('value', update_plot)

layout1 = column(widgetbox(select),p4)
show(layout1)
# curdoc().add_root(layout1)

# Use the following code to test in a notebook, comment out for transfer to live site
# Interactive features will not show in notebook
#output_notebook()
# show(layout)
#final_layout1 = column(p,p1,p2,layout)
#curdoc().add_root(final_layout1)


Loading BokehJS ...